In [1]:
import importlib

# List of required libraries
libraries = [
    "tensorflow", "scanpy", "pandas", "scipy", "scikit-learn",
    "louvain", "numpy", "h5py", "matplotlib", "seaborn"
]

# Check each library
for lib in libraries:
    try:
        module = importlib.import_module(lib)
        version = getattr(module, "__version__", "Unknown version")
        print(f"✅ {lib} is installed, version: {version}")
    except ImportError:
        print(f"❌ {lib} is NOT installed!")

✅ tensorflow is installed, version: 2.16.2
✅ scanpy is installed, version: 1.9.8
✅ pandas is installed, version: 2.2.2
✅ scipy is installed, version: 1.11.2
❌ scikit-learn is NOT installed!
✅ louvain is installed, version: 0.8.2
✅ numpy is installed, version: 1.26.4
✅ h5py is installed, version: 3.12.1
✅ matplotlib is installed, version: 3.8.4
✅ seaborn is installed, version: 0.13.2


In [2]:
"""Broadly useful python packages"""
import pandas as pd
import os
import numpy as np
import pickle
from copy import deepcopy
from shutil import move
import warnings

"""Machine learning and single cell packages"""
import sklearn.metrics as metrics
from sklearn.metrics import adjusted_rand_score as ari, normalized_mutual_info_score as nmi
import scanpy as sc
from anndata import AnnData

"""CarDEC Package"""
from CarDEC import CarDEC_API

2025-02-10 13:29:55.859356: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-02-10 13:29:55.859382: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2025-02-10 13:29:55.859390: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2025-02-10 13:29:55.859405: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-10 13:29:55.859414: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
"""Miscellaneous useful functions"""

def read_macaque(path):
    """A function to read and preprocess the macaque data"""
    adata = sc.read(path)
    sc.pp.filter_cells(adata, min_genes=0)
    sc.pp.filter_genes(adata, min_cells=30)
    
    adata = adata[adata.obs['n_genes'] < 2500, :]
    
    return(adata)

def purity_score(y_true, y_pred):
    """A function to compute cluster purity"""
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)

    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)

def find_resolution(adata_, n_clusters, random = 0): 
    adata = adata_.copy()
    obtained_clusters = -1
    iteration = 0
    resolutions = [0., 1000.]
    
    while obtained_clusters != n_clusters and iteration < 50:
        current_res = sum(resolutions)/2
        sc.tl.louvain(adata, resolution = current_res, random_state = random)
        labels = adata.obs['louvain']
        obtained_clusters = len(np.unique(labels))
        
        if obtained_clusters < n_clusters:
            resolutions[0] = current_res
        else:
            resolutions[1] = current_res
        
        iteration = iteration + 1
        
    return current_res

metrics_ = [ari, nmi, purity_score]

In the following cell, we read in the data.

In [4]:
"""Read and normalize the data"""
adata = read_macaque("macaque_bc.h5ad")

/opt/anaconda3/envs/cardec/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


Now, intialize the CarDEC class. Doing so will normalize the dataset. The results will be stored in an anndata object, referenced by CarDEC.dataset

In [5]:
"""Args: 
    1. adata is the dataframe to work on
    2. weights_dir: A directory in which to save weights for both the autoencoder pretrain step, and for the finetuned
    CarDEC model. Weights are also loaded from this directory. If the directory doesn't exist, then it will be 
    created from scratch.
    3. batch_key is the key in adata.obs that identifies the vector of cell batch assignments
    4. n_high_var is the number of features to treat as highly variable. These features drive clustering. The top
       n_high_var highly variable genes are identified with scanpy, using within batch variation
    5. LVG: If True, then denoise low variance features too. Else, only denoise the high variance genes.
"""

CarDEC = CarDEC_API(adata, weights_dir = "weights_dir/CarDEC_LVG Weights", batch_key = "sample", n_high_var = 2000, LVG = True)

/opt/anaconda3/envs/cardec/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:248: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_counts'] = number
/opt/anaconda3/envs/cardec/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:226: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby("mean_bin")["dispersions"]
/opt/anaconda3/envs/cardec/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:226: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby("m

## Fit the CarDEC Model

Now, build the model. If weights for the autoencoder do not exist in the weights directory, the autoencoder will be pretrained and its weights will be saved.

In [6]:
CarDEC.build_model(n_clusters=11, pretrain_epochs=10)

Pretrain weight index file not detected, pretraining autoencoder weights.

Epoch 000: Training Loss: 0.927, Validation Loss: 0.909, Time: 11.5 s
Epoch 001: Training Loss: 0.911, Validation Loss: 0.904, Time: 13.0 s
Epoch 002: Training Loss: 0.905, Validation Loss: 0.900, Time: 12.9 s
Epoch 003: Training Loss: 0.900, Validation Loss: 0.898, Time: 12.2 s
Epoch 004: Training Loss: 0.897, Validation Loss: 0.896, Time: 12.5 s
Epoch 005: Training Loss: 0.895, Validation Loss: 0.895, Time: 12.6 s
Epoch 006: Training Loss: 0.892, Validation Loss: 0.894, Time: 12.7 s
Epoch 007: Training Loss: 0.891, Validation Loss: 0.894, Time: 13.0 s
Epoch 008: Training Loss: 0.890, Validation Loss: 0.893, Time: 13.0 s
Epoch 009: Training Loss: 0.888, Validation Loss: 0.892, Time: 10.9 s

Training Completed
Total training time: 124.48 seconds
Autoencoder weights loaded.

Initializing cluster centroids using the louvain method.


/opt/anaconda3/envs/cardec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



 11 clusters detected. 


-----------------------CarDEC Architecture-----------------------



Model: "car_dec__model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sae (SAE)                       │ ?                      │       522,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Sequential)            │ (1, 32)                │       260,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Sequential)            │ (1, 2000)              │       262,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoderLVG (Sequential)         │ (1, 32)                │     2,062,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoderLVG (Sequential)         │ (1, 16083)             │     2,083,027 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ clustering (ClusteringLayer)    │ (1, 11)                │           352 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

Model: "decoderLVG"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ decoderLVG0 (Dense)             │ (1, 128)               │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ outputLVG (Dense)               │ (1, 16083)             │     2,074,707 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,083,027 (7.95 MB)

 Trainable params: 2,083,027 (7.95 MB)

 Non-trainable params: 0 (0.00 B)

Now, call the make_inference method to finetune CarDEC. Doing so will finetune the model, and then produce denoised features on the zscore scale. If weights for the full model are already saved in the weights directory, these weights will be loaded, rather than training the full model.

In [7]:
CarDEC.make_inference()

CarDEC Model Weights not detected. Training full model.



ValueError: Exception encountered when calling Sequential.call().

[1mInputs to a layer should be tensors. Got 'None' (of type <class 'NoneType'>) as input for layer 'encoder0'.[0m

Arguments received by Sequential.call():
  • inputs=None
  • training=None
  • mask=None

To get denoised features on the count scale, call the model_counts method.

In [ ]:
CarDEC.model_counts()

As mentioned before, the output is accessed via CarDEC.dataset. Let's look at the output structure.

In [ ]:
print("The overall structure of the output is: \n")
print(CarDEC.dataset)

CarDEC.dataset.X #The main layer of the output object contains the original counts
CarDEC.dataset.layers['denoised'] #These are the denoised features, on the zscore scale
CarDEC.dataset.layers['denoised counts'] #These are the denoised features, on the count scale
CarDEC.dataset.var['Variance Type'] #This is a vector that informs which genes are high variance and which are low variance
CarDEC.dataset.obsm['embedding'] #This is the CarDEC low-dimensional embedding after finetuning.
CarDEC.dataset.obsm['precluster denoised'] #This is the matrix of feature zscores denoised with the pretrained autoencoder.
CarDEC.dataset.obsm['precluster embedding'] #This is the latent embedding from the pretrained autoencoder.

"""Example, this is how to get the matrix of denoised counts for only high variance genes"""
HVG_denoised = deepcopy(CarDEC.dataset.layers['denoised counts'][:, CarDEC.dataset.var['Variance Type'] == 'HVG'])

"""Example, this is how to get the matrix of denoised counts for only low variance genes"""
LVG_denoised = deepcopy(CarDEC.dataset.layers['denoised counts'][:, CarDEC.dataset.var['Variance Type'] == 'LVG'])

## Working with the embedding and cluster assignments

Here, I demonstrate how to access the latent embedding of CarDEC and how to use it for UMAP visualization. I also demonstrate how to get the CarDEC cluster assignments.

In [ ]:
"""Get the predicted cluster assignments and compute cluster accuracy metrics"""

embedded = deepcopy(CarDEC.dataset.obsm['embedding']) #The latent embedding numpy array

q = deepcopy(CarDEC.dataset.obsm['cluster memberships']) #The cluster membership numpy array
labels = np.argmax(q, axis=1)
labels = [str(x) for x in labels]

true_celltype = list(CarDEC.dataset.obs['cluster']) #Note: all obs properties from the inputted adata are inherited by the output

print("CarDEC Clustering Results")
ARI, NMI, Purity = [metric(CarDEC.dataset.obs['cluster'], labels) for metric in metrics_]

print ("ARI = {0:.4f}".format(ARI)) 
print ("NMI = {0:.4f}".format(NMI)) 
print ("Purity = {0:.4f}".format(Purity))

"""Create a scanpy AnnData object with the latent embedding as the matrix, to perform scanpy UMAP embedding"""
formatting = AnnData(embedded)
formatting.obs["cell_type"] = list(CarDEC.dataset.obs['cluster'])
formatting.obs["predicted"] = list(labels)
formatting.obs["sample"] = list(CarDEC.dataset.obs['sample'])
formatting.obs["macaque_id"] = list(CarDEC.dataset.obs['macaque_id'])

sc.pp.neighbors(formatting, n_neighbors = 15, use_rep = 'X')
sc.tl.umap(formatting)
sc.pl.umap(formatting, color = ["predicted", "cell_type", "sample", "macaque_id"], return_fig = True)
print("Done")

In [ ]:
"""Get the predicted labels and compute adjusted rand score for the precluster embedding"""

preclust_emb = deepcopy(CarDEC.dataset.obsm['precluster embedding'])

formatting = AnnData(preclust_emb)
sc.pp.neighbors(formatting, n_neighbors = 15, use_rep = 'X')
res = find_resolution(formatting, 11)
sc.tl.louvain(formatting, resolution = res)

print(str(len(np.unique(labels))) + " Clusters Detected")

labels = formatting.obs['louvain']
type_strings = list(CarDEC.dataset.obs['cluster'])

ARI, NMI, Purity = [metric(CarDEC.dataset.obs['cluster'], labels) for metric in metrics_]

print("Pretrained Autoencoder Clustering Results")
print ("ARI = {0:.4f}".format(ARI))
print ("NMI = {0:.4f}".format(NMI))
print ("Purity = {0:.4f}".format(Purity))

formatting.obs["cell_type"] = list(CarDEC.dataset.obs['cluster'])
formatting.obs["predicted"] = list(labels)
formatting.obs["sample"] = list(CarDEC.dataset.obs['sample'])
formatting.obs["region"] = list(CarDEC.dataset.obs['region'])
formatting.obs["macaque_id"] = list(CarDEC.dataset.obs['macaque_id'])
sc.tl.umap(formatting)
sc.pl.umap(formatting, color = ["predicted", "cell_type", "sample", "macaque_id"], return_fig=True)
print("Done")

## Working with the denoised counts

Here I work with the denoised counts. I demonstrate the use of the denoised counts for UMAP embedding and louvain clustering with scanpy.

In [ ]:
"""Assessing denoised Counts"""

temporary = AnnData(deepcopy(CarDEC.dataset.layers['denoised counts']))
temporary.obs = CarDEC.dataset.obs
temporary.obs['cell_type'] = temporary.obs['cluster']

sc.pp.normalize_total(temporary)
sc.pp.log1p(temporary)
sc.pp.scale(temporary)

sc.tl.pca(temporary, svd_solver='arpack')
sc.pp.neighbors(temporary, n_neighbors = 15)

res = find_resolution(temporary, 11)
sc.tl.louvain(temporary, resolution = res)
temporary.obs['cluster assignment'] = temporary.obs['louvain']

sc.tl.umap(temporary)
sc.pl.umap(temporary, color = ["cell_type", "cluster assignment", "sample", "macaque_id"], return_fig = True)

ARI, NMI, Purity = [metric(temporary.obs['cell_type'], temporary.obs['cluster assignment']) for metric in metrics_]

print("CarDEC Denoising Results using all denoised counts")
print ("ARI = {0:.4f}".format(ARI)) 
print ("NMI = {0:.4f}".format(NMI)) 
print ("Purity = {0:.4f}".format(Purity))

## Working with only the high variance denoised counts

In [ ]:
"""Assessing HVG denoised Counts"""

temporary = AnnData(deepcopy(CarDEC.dataset.layers['denoised counts'][:, CarDEC.dataset.var['Variance Type'] == 'HVG']))
temporary.obs = CarDEC.dataset.obs
temporary.obs['cell_type'] = temporary.obs['cluster']

sc.pp.normalize_total(temporary)
sc.pp.log1p(temporary)
sc.pp.scale(temporary)

sc.tl.pca(temporary, svd_solver='arpack')
sc.pp.neighbors(temporary, n_neighbors = 15)

res = find_resolution(temporary, 11)
sc.tl.louvain(temporary, resolution = res)
temporary.obs['cluster assignment'] = temporary.obs['louvain']

sc.tl.umap(temporary)
sc.pl.umap(temporary, color = ["cell_type", "cluster assignment", "sample", "macaque_id"], return_fig = True)

ARI, NMI, Purity = [metric(temporary.obs['cell_type'], temporary.obs['cluster assignment']) for metric in metrics_]

print("Clustering high variance denoised counts")
print ("ARI = {0:.4f}".format(ARI)) 
print ("NMI = {0:.4f}".format(NMI)) 
print ("Purity = {0:.4f}".format(Purity))

## Working with only the low variance denoised counts

In [ ]:
"""Assessing LVG denoised Counts"""

temporary = AnnData(deepcopy(CarDEC.dataset.layers['denoised counts'][:, CarDEC.dataset.var['Variance Type'] == 'LVG']))
temporary.obs = CarDEC.dataset.obs
temporary.obs['cell_type'] = temporary.obs['cluster']

sc.pp.normalize_total(temporary)
sc.pp.log1p(temporary)
sc.pp.scale(temporary)

sc.tl.pca(temporary, svd_solver='arpack')
sc.pp.neighbors(temporary, n_neighbors = 15)

res = find_resolution(temporary, 11)
sc.tl.louvain(temporary, resolution = res)
temporary.obs['cluster assignment'] = temporary.obs['louvain']

sc.tl.umap(temporary)
sc.pl.umap(temporary, color = ["cell_type", "cluster assignment", "sample", "macaque_id"], return_fig = True)

ARI, NMI, Purity = [metric(temporary.obs['cell_type'], temporary.obs['cluster assignment']) for metric in metrics_]

print("Clustering low variance denoised counts")
print ("ARI = {0:.4f}".format(ARI)) 
print ("NMI = {0:.4f}".format(NMI)) 
print ("Purity = {0:.4f}".format(Purity))

## Working with the denoised counts on the zscore scale

In [ ]:
"""Assessing denoised zscore features"""

temporary = AnnData(deepcopy(CarDEC.dataset.layers['denoised']))
temporary.obs = CarDEC.dataset.obs
temporary.obs['cell_type'] = temporary.obs['cluster']

sc.tl.pca(temporary, svd_solver='arpack')
sc.pp.neighbors(temporary, n_neighbors = 15)

res = find_resolution(temporary, 11)
sc.tl.louvain(temporary, resolution = res)
temporary.obs['cluster assignment'] = temporary.obs['louvain']

sc.tl.umap(temporary)
sc.pl.umap(temporary, color = ["cell_type", "cluster assignment", "sample", "macaque_id"], return_fig = True)

ARI, NMI, Purity = [metric(temporary.obs['cell_type'], temporary.obs['cluster assignment']) for metric in metrics_]

print("CarDEC Denoising Results using all denoised features")
print ("ARI = {0:.4f}".format(ARI)) 
print ("NMI = {0:.4f}".format(NMI)) 
print ("Purity = {0:.4f}".format(Purity))